# NenaParser: A parser for Nena Standard Text format

The goal of this parser is to parse texts written in the plaintext
[NENA markup format][nenamarkup] and deliver a structured
list of words and their features, as well as paragraphing and line 
marks, which can be stored in a data format such as [Text-Fabric][textfabric], 
[Text-as-Graph][textasgraph] or (less optimally), XML or other hierarchical 
structures. 

For the Nena Markup parser, we make use of [Sly][sly], a Python implementation 
of the lex/yacc type of parser generators.

[nenamarkup]: ../docs/nena_format.md
[sly]: https://sly.readthedocs.io/en/latest/
[textfabric]: https://github.com/annotation/text-fabric
[textasgraph]: https://www.balisage.net/Proceedings/vol19/print/Dekker01/BalisageVol19-Dekker01.html

In [3]:
import re
import json
from sly import Lexer, Parser
import unicodedata
from pprint import pprint

# prepare alphabet and punctuation standards for processing
alphabet_std = '../standards/alphabet.json'
punctuation_std = '../standards/punctuation.json' 

with open(alphabet_std, 'r') as infile:
    alphabet_data = {
        re.compile(data['decomposed_regex']):data 
            for data in json.load(infile)
    }
with open(punctuation_std, 'r') as infile:
    punct_data = {
        re.compile(data['regex']):data 
            for data in json.load(infile)
    }
    
# compile regexes for matching
alphabet_re = '|'.join(data['decomposed_regex'] for letter,data in alphabet_data.items())

punct_begin_re = '|'.join(
    data['regex'] for punct, data in punct_data.items()
        if data['position'] == 'begin'
)
punct_end_re = '|'.join(
    data['regex'] for punct, data in punct_data.items()
        if data['position'] == 'end'
)

## Testing Sly

In [9]:
class TestLexer(Lexer):
    tokens = {LETTER, SPACE}
    LETTER = '[A-Za-z]'
    SPACE = '\s'
    
class TestParser(Parser):
    
    debugfile = 'test_parser.out'
    tokens = TestLexer.tokens
    precedence = (
        ('left', SECOND),
        ('left', LETTER),
    )
    
    @_('word words')
    def words(self, p):
        return [p.word] + p.words
    
    @_('word')
    def words(self, p):
        return [p.word]
    
    @_('letters SPACE')
    def word(self, p):
        return {
            'letters': p.letters,
            'punct': p.SPACE,
        }
    
    @_('letters')
    def word(self, p):
        return {
            'letters': p.letters,
            'punct': '',
        }
    
    @_('LETTER letters')
    def letters(self, p):
        return [p.LETTER] + p.letters 
    
    @_('LETTER %prec SECOND')
    def letters(self, p):
        return [p[0]]
    
    
test_lexer = TestLexer()
test_parser = TestParser()
test_string = 'This is'
test_parser.parse(test_lexer.tokenize(test_string))

Parser debugging for TestParser written to test_parser.out


[{'letters': ['T', 'h', 'i', 's'], 'punct': ' '},
 {'letters': ['i', 's'], 'punct': ''}]

## Example Text

Below is a dummy text we can use to test the parsers on.

In [4]:
example = unicodedata.normalize('NFD', '''
dialect: Urmi_C
title: When Shall I Die?
encoding: UTF8
informant: Yulia Davudi
interviewer: Geoffrey Khan
place: +Hassar +Baba-čanɟa, N
transcriber: Geoffrey Khan
text_id: A32 

(1@0:00) xá-yuma "⁺malla ⁺Nasrádən" váyələ tíva ⁺ʾal-k̭èsa.ˈ xá mən-nášə 
⁺vàrəva,ˈ mə́rrə ⁺màllaˈ ʾátən ʾo-k̭ésa pràmut,ˈ bət-nàplət.ˈ mə́rrə <P: bŏ́ro> 
bàbaˈ ʾàtən=daˈ ⁺šúla lə̀tluxˈ tíyyət b-dìyyi k̭ítət.ˈ ⁺šúk̭ si-⁺bar-⁺šùlux.ˈ 
ʾána ⁺šūl-ɟànilə.ˈ náplən nàplən.ˈ (2@0:08) ⁺hàlaˈ ʾo-náša léva xíša xá 
⁺ʾəsrá ⁺pasulyày,ˈ ⁺málla bitáyələ drúm ⁺ʾal-⁺ʾàrra.ˈ bək̭yámələ ⁺bərxáṱələ 
⁺bàru.ˈ màraˈ ⁺maxlèta,ˈ ʾátən ⁺dílux ʾána bət-náplənva m-⁺al-ʾilàna.ˈ 
bas-tánili xázən ʾána ʾíman bət-mètən.ˈ ʾo-náša xzílə k̭at-ʾá ⁺màllaˈ hónu 
xáč̭č̭a ... ⁺basùrələˈ mə́rrə k̭àtuˈ ⁺maxlèta,ˈ mə̀drə,ˈ «GK: maxlèta?» ⁺rába 
⁺maxlèta.ˈ mə́rrə k̭at-ʾíman xmártux ⁺ṱlá ɟáhə ⁺ʾarṱàla,ˈ ʾó-yuma mètət.ˈ 
ʾó-yumət xmártux ⁺ṱlá ɟáhə ⁺ʾarṱàla,ˈ ʾó-yuma mètət.ˈ 

(3@0:16) ⁺málla múttəva ... ⁺ṱànaˈ ⁺yak̭úyra ⁺ʾal-xmàrta.ˈ ⁺ṱànaˈ mə́ndi 
⁺rába múttəva ⁺ʾal-xmàrtaˈ ʾu-xmàrtaˈ ⁺báyyava ʾask̭áva ⁺ʾùllul.ˈ
ʾu-bas-pòxa ⁺plə́ṱlə mənnó.ˈ ṱə̀r,ˈ ⁺riṱàla.ˈ ⁺málla mə́rrə ʾàha,ˈ ʾána dū́n
k̭arbúnə k̭a-myàta.ˈ (4@0:20) xáč̭č̭a=da sə̀k̭laˈ xa-xìta.ˈ ɟánu mudməxxálə
⁺ʾal-⁺ʾàrra.ˈ mə̀rrəˈ xína ⁺dā́n mòtila.ˈ ʾē=t-d-⁺ṱlàˈ ⁺málla mə̀tlə.ˈ nàšə,ˈ
 xuyravàtuˈ xə́šlun tílun mə̀rrunˈ ʾa mù-vadət? k̭a-mú=ivət ⁺tàmma?ˈ mə́rrə 
 xob-ʾána mìtən.ˈ lá bəxzáyətun k̭at-mìtən!ˈ lá mə́rrun ʾat-xàya!ˈ 
 hamzùməvət.ˈ bəšvák̭una ⁺tàmaˈ màraˈ xmàrələ,ˈ lélə ⁺parmùyə.ˈ
 ''')

## Structure of a text with NENA Markup

Below is a representation of the tree-like structure of a NENA standard text file. This is the structure that the parser must recognize and reproduce.

`+` is used to represent one or more elements.

```
text
  |
  metadata block
  |  |
  |  +attribute
  | 
  text block
    |
    +paragraph
      |   
      +line
        |
        +word
          |
          +letter
```

These items will be returned in the following Pythonic representation:

In [5]:
[ # text
    [ # metadata block
        ('dialect', 'Urmi_C'),
        ('title', 'When Shall I Die?'),
        ('encoding', 'UTF8'),
    ],
    [ # text block
        [ # paragraph
            [ # line
                ('number', '1'),
                ('timestamp', '0:00'),
                ('words', [
                        { # word
                            'text':'xá',
                            'begin':'',
                            'end':'-',
                            'lang':'NENA', 
                            'letters':('x','á'),
                        },
                        # ...
                        { # foreign word
                            'text':'bŏ́ro',
                            'begin':'<P:',
                            'end': '> ',
                            'lang': 'P', 
                            'letters':('b','ŏ́','r','o'),
                        }, 
                    ],
                ),
            ],
        ],
    ],
]

[[('dialect', 'Urmi_C'), ('title', 'When Shall I Die?'), ('encoding', 'UTF8')],
 [[[('number', '1'),
    ('timestamp', '0:00'),
    ('words',
     [{'text': 'xá',
       'begin': '',
       'end': '-',
       'lang': 'NENA',
       'letters': ('x', 'á')},
      {'text': 'bŏ́ro',
       'begin': '<P:',
       'end': '> ',
       'lang': 'P',
       'letters': ('b', 'ŏ́', 'r', 'o')}])]]]]

## Lexer

The parser needs as its input 'tokens', which are predefined units of characters. These are provided by the 'lexer'. In Sly (and Ply), tokens are defined as regular expressions, of which the matching string is returned as the token value. If the token is defined as a function (with its regular expression as argument to the `@_` decorator), then the returned value (among other things) can be manipulated. For more detailed information, [see the documentation][slydocs].

[slydocs]: https://sly.readthedocs.io/en/latest/sly.html

In [6]:
class NenaLexer(Lexer):
    
    def error(self, t):
        """Give warning for bad characters"""
        print(f"Illegal character {repr(t.value[0])} @ index {self.index}")
        self.index += 1
    
    # set of token names as required by
    # the Lexer class
    tokens = {
        LETTER, PUNCT_BEGIN, PUNCT_END, NEWLINES,
        NEWLINE, NEWLINES, ATTRIBUTE, 
        LANG_START, LANG_END, FOREIGN_LETTER,
        LINENUMBER, TIMESTAMP, 
        SPEAKER_START, SPEAKER_END,
    }

    # Attribute starts key and colon. Returns 2-tuple (key, value).
    @_(r'[a-z][a-z0-9_]+: .*')
    def ATTRIBUTE(self, t):
        t.value = tuple(t.value.split(': '))
        return t
    
    @_(r'\(\d+\@\d:\d+\)\s*', 
       r'\(\d+\)\s*')
    def LINESTAMP(self, t):
        t.value = t.value.strip()
        return t

    LETTER = alphabet_re
    PUNCT_BEGIN = punct_begin_re
    PUNCT_END = punct_end_re
    NEWLINES = r'\n\s*\n\s*' # i.e. marks text-blocks
    NEWLINE = r'\n'
    
    # Language markers are ASCII letter strings 
    # surrounded by angle brackets.
    LANG_START = r'<[A-Za-z]+:'
    LANG_END = r'>'
    
    @_(r'»')
    def SPEAKER_END(self, t):
        return t
    
    @_(r'«[a-z\sA-Z]+: ')
    def SPEAKER_START(self, t):
        t.value = re.match(r'«([a-z\sA-Z]+): ', t.value).group(1)
        return t
    
    # NB: tokens evaluated in order of appearance here
    # thus foreign string matched lastly
    FOREIGN_LETTER = r'[a-zA-ZðÐɟəƏɛƐʾʿθΘ][\u0300-\u033d]*'

In [7]:
# demonstration of output results of lexer, to be used by parser below
lexer = NenaLexer()

tokens = [(tok.type, tok.value) for tok in lexer.tokenize(example)]

In [8]:
pprint(tokens[:100])

[('NEWLINE', '\n'),
 ('ATTRIBUTE', ('dialect', 'Urmi_C')),
 ('NEWLINE', '\n'),
 ('ATTRIBUTE', ('title', 'When Shall I Die?')),
 ('NEWLINE', '\n'),
 ('ATTRIBUTE', ('encoding', 'UTF8')),
 ('NEWLINE', '\n'),
 ('ATTRIBUTE', ('informant', 'Yulia Davudi')),
 ('NEWLINE', '\n'),
 ('ATTRIBUTE', ('interviewer', 'Geoffrey Khan')),
 ('NEWLINE', '\n'),
 ('ATTRIBUTE', ('place', '+Hassar +Baba-čanɟa, N')),
 ('NEWLINE', '\n'),
 ('ATTRIBUTE', ('transcriber', 'Geoffrey Khan')),
 ('NEWLINE', '\n'),
 ('ATTRIBUTE', ('text_id', 'A32 ')),
 ('NEWLINES', '\n\n'),
 ('LINESTAMP', '(1@0:00)'),
 ('LETTER', 'x'),
 ('LETTER', 'á'),
 ('PUNCT_END', '-'),
 ('LETTER', 'y'),
 ('LETTER', 'u'),
 ('LETTER', 'm'),
 ('LETTER', 'a'),
 ('PUNCT_END', ' '),
 ('PUNCT_BEGIN', '"'),
 ('PUNCT_BEGIN', '⁺'),
 ('LETTER', 'm'),
 ('LETTER', 'a'),
 ('LETTER', 'l'),
 ('LETTER', 'l'),
 ('LETTER', 'a'),
 ('PUNCT_END', ' '),
 ('PUNCT_BEGIN', '⁺'),
 ('LETTER', 'N'),
 ('LETTER', 'a'),
 ('LETTER', 's'),
 ('LETTER', 'r'),
 ('LETTER', 'á'),
 ('LETT

### The parser

The parser processes the tokens provided by the lexer, and tries to combine them into structured units. Those units are defined in the methods of the `NenaParser` class, with the patterns passed as arguments to the `@_` decorator.

The top unit (in this case, `text`) is returned as the result of the parsing.

In [10]:
class NenaParser(Parser):    
    
    debugfile = 'nena_parser.out'
    tokens = NenaLexer.tokens

    precedence = (
        ('left', SECOND),
        ('left', LETTER,  FOREIGN_LETTER)
    )
    
    @_('word words')
    def text(self, p):
        return [p.word] + p.words
    
    @_('word')
    def words(self, p):
        return [p.word]
    
    # build words with their associated punctuators
    @_('begins letters ends')
    def word(self, p):
        print(p.letters)
        return {
            'word': ''.join(p.letters),
            'letters': p.letters,
            'punct_begin': p.begins,
            'punct_end': p.ends,
        }
    
    @_('begins letters')
    def word(self, p):
        print(p.letters)
        return {
            'word': ''.join(p.letters),
            'letters': p.letters,
            'punct_begin': p.begins,
            'punct_end': '',
        }
    
    @_('letters')
    def word(self, p):
        print(p.letters)
        return {
            'word': ''.join(p.letters),
            'letters': p.letters,
            'punct_begin': '',
            'punct_end': '',
        }
    
    @_('PUNCT_END ends')
    def ends(self, p):
        return [p.PUNCT_END] + p.ends
    
    @_('PUNCT_END')
    def ends(self, p):
        return [p.PUNCT_END]
       
    @_('PUNCT_BEGIN begins')
    def begins(self, p):
        return [p.PUNCT_BEGIN] + p.begins
    
    @_('PUNCT_BEGIN')
    def begins(self, p):
        return [p.PUNCT_BEGIN]
        
    @_('LETTER letters', 
       'FOREIGN_LETTER letters')
    def letters(self, p):
        return [p[0]] + p[1]
    
    @_('LETTER %prec SECOND', 
       'FOREIGN_LETTER %prec SECOND')
    def letters(self, p):
        return [p[0]]
                
nena_test = unicodedata.normalize('NFD', 'xá-yuma "⁺m') #alla ⁺Nasrádən" váyələ tíva')
         
# demonstration of output results of parser, to be used by generate_TF loop
parser = NenaParser()
parser.parse(lexer.tokenize(nena_test))

['m']


Parser debugging for NenaParser written to nena_parser.out
sly: Syntax error at line 1, token=PUNCT_END
sly: Syntax error at line 1, token=PUNCT_END
sly: Parse error in input. EOF


In [2]:
list(lexer.tokenize(nena_test))

NameError: name 'lexer' is not defined

In [ ]:
    @_('PUNCT_END ends')
    def ends(self, p):
        return [p.PUNCT_END] + p.ends
    
    @_('PUNCT_END')
    def ends(self, p):
        return [p.PUNCT_END]
       
    @_('PUNCT_BEGIN begins')
    def begins(self, p):
        return [p.PUNCT_BEGIN] + p.begins
    
    @_('PUNCT_BEGIN')
    def begins(self, p):
        return [p.PUNCT_BEGIN]

In [ ]:
#    

#     precedence = (
#         ('left', CLUSTER),
#         ('left', CLUSTERED),
#     )
    
#     @_('letters SPACE', 
#        'letters %prec CLUSTERED')
#     def word(self, p):

    
#     @_('letters letters %prec CLUSTER')
#     def letters(self, p):
#         return p[0] + p[1]

In [97]:
def get_string_data(string, str_data):
    """Match a string to its correlated data.
    
    A "string" can be a letter or a punctuator.
    
    Args:
        string: str to be matched with data
        str_data: dict of data for strings where each key
            is a compiled regex object that matches
            a given letter to a dict of data (the values).
            
    Returns:
        dict of letter data
    """
    for str_pattern, data in str_data.items():
        if str_pattern.match(string):
            return data

In [5]:
class NenaParser(Parser):

    # Get the token list from the lexer (required)
    tokens = NenaLexer.tokens
    

In [ ]:
# demonstration of output results of parser, to be used by generate_TF loop
parser = NenaParser()
parser.parse(lexer.tokenize(parser_test))

### Parser output

The parser prints a warning that there were shift/reduce conflicts, probably caused by ambiguous whitespace. That is not a problem (although not very elegant, ideally it should be fixed). The parser resolves the conflicts automatically.

The output of the example text shows that the parser succeeded to parse it, and structure it into heading, paragraphs, lines and morphemes, with the features stored in the Morpheme object.

## Testing with Real Texts

In [ ]:
from pathlib import Path

In [ ]:
# paths
data_dir = Path('../nena/0.01')
dialect_dirs = list(Path(data_dir).glob('*'))

### Run Parse On All Texts

In [ ]:
name2parsed = {}
name2text = {}
not_parsed = []

ignore = [
    #'The Adventures Of Two Brothers.nena', # FIX BY MOVING UNEMPHASIZED OUT
]

for dialect in dialect_dirs:
    print(f'--Dialect {dialect}--')
    print()
    for file in sorted(dialect.glob('*.nena')):
        
        if file.name in ignore:
            print('SKIPPING:', file.name, '\n')
            not_parsed.append(file)
            continue
        
        with open(file, 'r') as infile:
            text = infile.read()
            name2text[file.name] = text
            print(f'trying: {file.name}')
            parseit = parser.parse(lexer.tokenize(text))
            print(f'\t√')
            name2parsed[file.name] = parseit
                
print(len(name2parsed), 'parsed...')
print(len(not_parsed), 'not parsed...')

In [ ]:
#name2text['A Man Called Čuxo.nena'][7100:7120]